In [2]:
# Imports
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn import datasets

import importlib.util
spec = importlib.util.spec_from_file_location("DecisionTree", "../DecisionTree/decision_tree.py")
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)

DecisionTree = module.DecisionTree

In [3]:


# Data preparation
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [4]:
# Helper functions
def bootstrap_sample(X, y):
    n_samples = X.shape[0]
    idxs = np.random.choice(n_samples, n_samples, replace=True)
    return X[idxs], y[idxs]

def most_common_label(y):
    counter = Counter(y)
    return counter.most_common(1)[0][0]

def accuracy(y_true, y_pred):
    return np.sum(y_true == y_pred) / len(y_true)

In [7]:
class RandomForest:
    def __init__(self, n_trees=10, min_samples_split=2, max_depth=100, n_feats=None):
        self.n_trees = n_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(
                min_samples_split=self.min_samples_split,
                max_depth=self.max_depth,
                n_feats=self.n_feats,
            )
            X_samp, y_samp = bootstrap_sample(X, y)
            tree.fit(X_samp, y_samp)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(tree_preds, 0, 1)
        y_pred = [most_common_label(tree_pred) for tree_pred in tree_preds]
        return np.array(y_pred)

In [8]:
# classifier
clf = RandomForest()

# fit data and make prediction
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
acc = accuracy(y_test, y_pred)

print("Random Forest accuracy:", acc)

Random Forest accuracy: 0.9473684210526315
